In [ ]:
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pickle
from datetime import datetime
from mlflow.models import infer_signature
import  mlflow
from sklearn.metrics import precision_recall_fscore_support as score
import mlflow
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from dataprep.eda import create_report
from plotly.subplots import make_subplots
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer , SimpleImputer
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA, KernelPCA, FastICA
from sklearn.manifold import TSNE, MDS
import umap.umap_ as umap
import prince  # Multiple Correspondence Analysis
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
import missingno as msno
from feature_engine.outliers import OutlierTrimmer
from category_encoders import TargetEncoder, WOEEncoder, CatBoostEncoder
import sweetviz as sv
from datetime import datetime
import dataprep.eda as dpeda
from statsmodels.stats.outliers_influence import OLSInfluence
import statsmodels.api as sm
import AdvancedRelationshipAnalysis 
import json
from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import *
from sklearn.linear_model import LogisticRegression, LinearRegression
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *
import xgboost as xgb
import joblib
import optuna
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import (
    train_test_split, 
    StratifiedKFold, 
    cross_val_score
)
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import *


In [12]:
class AdvancedEDA:
    def __init__(self, df, target_col=None):
        """
        Initialize Advanced EDA class

        Parameters:
        df: pandas DataFrame
        target_col: str, target column name for supervised analysis
        """
        self.df = df.copy()
        self.target_col = target_col
        self.numeric_cols = df.select_dtypes(include=[np.number]).columns
        self.categorical_cols = df.select_dtypes(exclude=[np.number]).columns
        
    def basic_feature_cleaning(
        self,
        nan_threshold=0.5,
        correlation_threshold=0.95,
        variance_threshold=0.01,
        outlier_method='IQR',
        dominant_category_threshold=0.95,
        verbose=True,
    ):
        """
        Nettoyage des colonnes avancé avec suppression des colonnes inutiles ou redondantes.
        """
        df_cleaned = self.df
        print(df_cleaned.columns)
        print(len(df_cleaned.columns))
        
        num_cols = [col for col in self.numeric_cols if col in df_cleaned.columns]

        # # 7. Suppression des outliers (IQR ou Z-score)
        # if outlier_method == 'IQR':
        #     if verbose:
        #         print("Suppression des outliers avec la méthode IQR...")
        #     Q1 = df_cleaned[num_cols].quantile(0.25)
        #     Q3 = df_cleaned[num_cols].quantile(0.75)
        #     IQR = Q3 - Q1
        #     lower_bound = Q1 - 1.5 * IQR
        #     upper_bound = Q3 + 1.5 * IQR
        #     df_cleaned = df_cleaned[~((df_cleaned[num_cols] < lower_bound) | (df_cleaned[num_cols] > upper_bound)).any(axis=1)]
        
        # elif outlier_method == 'Z-score':
        #     if verbose:
        #         print("Suppression des outliers avec la méthode Z-score...")
        #     from scipy.stats import zscore
        #     z_scores = np.abs(zscore(df_cleaned[num_cols]))
        #     df_cleaned = df_cleaned[(z_scores < 3).all(axis=1)]
        # # 1. Supprimer les colonnes contenant "id" dans leur nom
        if verbose:
            print("Suppression des colonnes contenant 'id' dans leur nom...")
        id_cols = [col for col in df_cleaned.columns if 'id' in col.lower()]
        print(id_cols)
        df_cleaned = df_cleaned.drop(columns=id_cols)

        # 2. Supprimer les colonnes avec trop de NaN
        if verbose:
            print("Suppression des colonnes avec trop de valeurs manquantes...")
        nan_cols = df_cleaned.columns[df_cleaned.isnull().mean() > nan_threshold]
        df_cleaned = df_cleaned.drop(columns=nan_cols)

        # 3. Supprimer les colonnes contenant une seule valeur unique
        if verbose:
            print("Suppression des colonnes avec une seule valeur unique...")
        unique_val_cols = [col for col in df_cleaned.columns if df_cleaned[col].nunique() <= 1]
        print(unique_val_cols)
        df_cleaned = df_cleaned.drop(columns=unique_val_cols)
        unique_val_cols2 = [col for col in df_cleaned.columns if len(list(df_cleaned[col].value_counts())) == 1]
        print(unique_val_cols2)
        df_cleaned = df_cleaned.drop(columns=unique_val_cols2)

        # 4. Supprimer les colonnes catégoriques avec une catégorie dominante
        if verbose:
            print("Suppression des colonnes avec une catégorie dominante...")
        dom_cat_cols = [
            col for col in df_cleaned.select_dtypes(exclude=[np.number]).columns
            if (df_cleaned[col].value_counts(normalize=True).max() > dominant_category_threshold)
        ]
        df_cleaned = df_cleaned.drop(columns=dom_cat_cols)

        # 5. Suppression des colonnes avec faible variance
        if verbose:
            print("Suppression des colonnes avec faible variance...")
        num_cols = df_cleaned.select_dtypes(include=[np.number]).columns
        if len(num_cols) > 0:
            var_selector = VarianceThreshold(threshold=variance_threshold)
            try:
                var_selector.fit(df_cleaned[num_cols])
                low_variance_cols = num_cols[~var_selector.get_support()]
                df_cleaned = df_cleaned.drop(columns=low_variance_cols, errors="ignore")
            except Exception as e:
                if verbose:
                    print(f"Erreur dans la sélection par variance : {e}")
        num_cols = [col for col in self.numeric_cols if col in df_cleaned.columns]

        # 6. Supprimer les colonnes fortement corrélées
        if verbose:
            print("Suppression des colonnes fortement corrélées...")
        corr_matrix = df_cleaned[num_cols].corr().abs()
        upper_triangle = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
        to_drop = [
            column for column in corr_matrix.columns
            if any(corr_matrix[column][upper_triangle[:, corr_matrix.columns.get_loc(column)]] > correlation_threshold)
        ]
        df_cleaned = df_cleaned.drop(columns=to_drop, errors="ignore")

        num_cols = [col for col in self.numeric_cols if col in df_cleaned.columns]

        # # 7. Suppression des outliers (IQR ou Z-score)
        # if outlier_method == 'IQR':
        #     if verbose:
        #         print("Suppression des outliers avec la méthode IQR...")
        #     Q1 = df_cleaned[num_cols].quantile(0.25)
        #     Q3 = df_cleaned[num_cols].quantile(0.75)
        #     IQR = Q3 - Q1
        #     lower_bound = Q1 - 1.5 * IQR
        #     upper_bound = Q3 + 1.5 * IQR
        #     df_cleaned = df_cleaned[~((df_cleaned[num_cols] < lower_bound) | (df_cleaned[num_cols] > upper_bound)).any(axis=1)]
        
        # elif outlier_method == 'Z-score':
        #     if verbose:
        #         print("Suppression des outliers avec la méthode Z-score...")
        #     from scipy.stats import zscore
        #     z_scores = np.abs(zscore(df_cleaned[num_cols]))
        #     df_cleaned = df_cleaned[(z_scores < 3).all(axis=1)]

        # Mise à jour des colonnes après nettoyage
        self.numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns
        self.categorical_cols = df_cleaned.select_dtypes(exclude=[np.number]).columns

        if verbose:
            print("Nettoyage des caractéristiques terminé.")
            print(f"Colonnes restantes : {df_cleaned.columns.tolist()}")

        return df_cleaned
    def advanced_feature_selection(
        self,
        methods=["basic", "mutual_info", "rfe", "chi2"],
        nan_threshold=0.5,
        correlation_threshold=0.95,
        variance_threshold=0.01,
        top_k_features=10,
        verbose=True,
    ):
        """
        Perform comprehensive feature selection using multiple techniques.
    
        Parameters:
        -----------
        methods : list, optional (default=['basic', 'mutual_info', 'rfe', 'chi2'])
            List of feature selection methods to apply
        nan_threshold : float, optional (default=0.5)
            Threshold for dropping columns with too many NaN values
        correlation_threshold : float, optional (default=0.95)
            Threshold for dropping highly correlated features
        variance_threshold : float, optional (default=0.01)
            Threshold for dropping low variance features
        top_k_features : int, optional (default=10)
            Number of top features to select in advanced methods
        verbose : bool, optional (default=True)
            Whether to print detailed information about feature selection
    
        Returns:
        --------
        pandas.DataFrame
            DataFrame with selected features
        """
        # Create a copy of the DataFrame to avoid modifying the original
        df_selected = self.df.copy()
        target_col = self.target_col
    
        # Validate target column
        if target_col and target_col not in df_selected.columns:
            raise ValueError(f"Target column '{target_col}' not found in DataFrame")
    
        # Preprocessing for feature selection methods
        def preprocess_data(df_input):
            """Preprocess data for feature selection methods"""
            # Handle categorical variables
            df_processed = pd.get_dummies(df_input, drop_first=True)
            
            # Separate features and target
            if target_col and target_col in df_processed.columns:
                X = df_processed.drop(columns=[target_col])
                y = df_processed[target_col]
            else:
                X, y = df_processed, None
            
            return X, y
    
        # 1. Basic Feature Selection
        if "basic" in methods:
            # Remove columns with too many NaN values
            df_selected = df_selected.loc[:, df_selected.isnull().mean() <= nan_threshold]
    
            # Remove ID-like columns
            id_cols = [col for col in df_selected.columns if "id" in col.lower()]
            df_selected = df_selected.drop(columns=id_cols, errors='ignore')
    
            # Remove single-value categorical columns
            cat_cols = [col for col in df_selected.select_dtypes(exclude=[np.number]).columns 
                        if df_selected[col].nunique() <= 1]
            df_selected = df_selected.drop(columns=cat_cols, errors='ignore')
    
            # Remove low-variance numerical columns
            num_cols = df_selected.select_dtypes(include=[np.number]).columns
            if len(num_cols) > 0:
                try:
                    var_selector = VarianceThreshold(threshold=variance_threshold)
                    var_selector.fit(df_selected[num_cols])
                    
                    # Identify and remove low variance columns
                    low_variance_cols = num_cols[~var_selector.get_support()]
                    df_selected = df_selected.drop(columns=low_variance_cols, errors='ignore')
                except Exception as e:
                    if verbose:
                        print(f"Variance threshold filtering error: {e}")
    
            # Remove highly correlated features
            try:
                corr_matrix = df_selected.select_dtypes(include=[np.number]).corr().abs()
                upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
                
                # Find columns to drop based on correlation
                to_drop = [column for column in upper.columns 
                           if any(upper[column] > correlation_threshold)]
                df_selected = df_selected.drop(columns=to_drop, errors='ignore')
            except Exception as e:
                if verbose:
                    print(f"Correlation-based feature selection error: {e}")
    
        # Prepare data for advanced methods
        X, y = preprocess_data(df_selected)
        
        # Ensure we have features to select from
        if X.empty:
            if verbose:
                print("No features left after basic selection!")
            return df_selected
    
        # Set of features to keep
        selected_features = set(X.columns)
    
        # 2. Mutual Information Feature Selection
        if "mutual_info" in methods and target_col and y is not None:
            try:
                # Choose appropriate mutual information method based on target type
                mi_func = (mutual_info_classif if y.dtype == 'object' or y.dtype == 'category'
                           else mutual_info_regression)
                
                # Compute mutual information scores
                mi_scores = mi_func(X, y)
                mi_df = pd.DataFrame({
                    "feature": X.columns, 
                    "mutual_info": mi_scores
                }).sort_values("mutual_info", ascending=False)
                
                # Select top k features
                mi_features = set(mi_df.head(min(top_k_features, len(mi_df)))['feature'])
                
                # Intersect with existing selected features
                selected_features &= mi_features
                
                if verbose:
                    print("Top features by Mutual Information:")
                    print(mi_df.head(min(top_k_features, len(mi_df))))
            except Exception as e:
                if verbose:
                    print(f"Mutual Information selection failed: {e}")
    
        # 3. Recursive Feature Elimination (RFE)
        if "rfe" in methods and target_col and y is not None:
            try:
                # Choose appropriate estimator based on target type
                estimator = (LogisticRegression(max_iter=1000) if y.dtype == 'object' or y.dtype == 'category'
                             else LinearRegression())
                
                # Perform RFE
                selector = RFE(estimator, n_features_to_select=min(top_k_features, X.shape[1]))
                selector.fit(X, y)
                
                # Get RFE selected features
                rfe_features = set(X.columns[selector.support_])
                
                # Intersect with existing selected features
                selected_features &= rfe_features
                
                if verbose:
                    print("\nTop features by RFE:")
                    print(X.columns[selector.support_])
            except Exception as e:
                if verbose:
                    print(f"Recursive Feature Elimination failed: {e}")
    
        # 4. Chi-Squared Feature Selection
        if "chi2" in methods and target_col and y is not None:
            try:
                # Chi-squared only works for classification problems
                if y.dtype == 'object' or y.dtype == 'category':
                    # Encode target variable
                    le = LabelEncoder()
                    y_encoded = le.fit_transform(y)
                    
                    # Scale features
                    scaler = StandardScaler()
                    X_scaled = scaler.fit_transform(X)
                    
                    # Select top k features
                    selector = SelectKBest(chi2, k=min(top_k_features, X.shape[1]))
                    selector.fit(X_scaled, y_encoded)
                    
                    # Get chi2 selected features
                    chi2_features = set(X.columns[selector.get_support()])
                    
                    # Intersect with existing selected features
                    selected_features &= chi2_features
                    
                    if verbose:
                        print("\nTop features by Chi-Squared Test:")
                        print(chi2_features)
                else:
                    if verbose:
                        print("Chi-Squared is not applicable for continuous target variables.")
            except Exception as e:
                if verbose:
                    print(f"Chi-Squared selection failed: {e}")
    
        # # Finalize feature selection
        # final_features = list(selected_features)
        
        # # Add target column back if it exists
        # if target_col:
        #     final_features.append(target_col)
        
        # # Return final dataframe with selected features
        # df_final = self.df[final_features]
        
        # # Update numeric and categorical columns
        # self.numeric_cols = df_final.select_dtypes(include=[np.number]).columns
        # self.categorical_cols = df_final.select_dtypes(exclude=[np.number]).columns
        
        # if verbose:
        #     print("\nFinal Selected Features:")
        #     print(final_features)
        
        # return df_final 
        # Finalize feature selection
        final_features = list(selected_features)
        
        # Ensure selected features exist in the original DataFrame
        final_features = [feature for feature in final_features if feature in self.df.columns]
        
        # Add target column back if it exists
        if target_col and target_col not in final_features:
            final_features.append(target_col)
        
        # Safely slice the DataFrame
        try:
            df_final = self.df[final_features]
        except KeyError as e:
            missing_features = list(set(final_features) - set(self.df.columns))
            if verbose:
                print(f"Warning: Missing features: {missing_features}")
            final_features = [f for f in final_features if f in self.df.columns]
            df_final = self.df[final_features]
        
        # Update numeric and categorical columns
        self.numeric_cols = df_final.select_dtypes(include=[np.number]).columns
        self.categorical_cols = df_final.select_dtypes(exclude=[np.number]).columns
        
        if verbose:
            print("\nFinal Selected Features:")
            print(final_features)
        
        return df_final

    def simple_statistical_imputation(self):
        """
        Perform simple imputation using:
        - mean or median based on Kolmogorov-Smirnov test for numerical variables
        - most frequent value for categorical variables
        
        Returns:
        DataFrame: Imputed dataframe
        dict: Dictionary containing the strategy used for each column
        """
        self.df = self.basic_feature_cleaning().copy()
        #self.df = self.advanced_feature_selection(methods=["basic", "rfe", "chi2"])
        df_imputed = self.df.copy()
        self.numeric_cols = df_imputed.select_dtypes(include=[np.number]).columns
        self.categorical_cols = df_imputed.select_dtypes(exclude=[np.number]).columns
        # imputation_strategies = {}
        
        # Handle numerical columns
        for col in self.numeric_cols:
            print(col)
            # Get non-null values for the column
            non_null_values = self.df[col].dropna()
            
            if len(non_null_values) > 0:
                # Perform Kolmogorov-Smirnov test against normal distribution
                _, p_value = stats.kstest(
                    stats.zscore(non_null_values), 
                    'norm'
                )
                
                # Choose strategy based on p-value
                if p_value < 0.05:
                    # Not normally distributed - use median
                    strategy = 'median'
                else:
                    # Normally distributed - use mean
                    strategy = 'mean'
                
                # Apply imputation
                imputer = SimpleImputer(strategy=strategy)
                df_imputed[col] = imputer.fit_transform(df_imputed[[col]])
                
                # # Store the strategy used
                # imputation_strategies[col] = {
                #     'strategy': strategy,
                #     'p_value': p_value,
                #     'fill_value': df_imputed[col].iloc[0] if strategy == 'mean' 
                #                 else df_imputed[col].median(),
                #     'missing_count': self.df[col].isnull().sum(),
                #     'missing_percentage': (self.df[col].isnull().sum() / len(self.df)) * 100
                # }
        
        # Handle categorical columns with mode imputation
        if len(self.categorical_cols) > 0:
            for col in self.categorical_cols:
                print(col)

                # Get mode (most frequent value)
                mode_value = self.df[col].mode()[0]
                
                # Apply mode imputation
                df_imputed[col] = self.df[col].fillna(mode_value)
                
                # Calculate frequency of the most common value
                value_counts = self.df[col].value_counts(normalize=True)
                most_common_freq = value_counts.iloc[0] if not value_counts.empty else 0
                
                # Store the strategy and statistics
                # imputation_strategies[col] = {
                #     'strategy': 'most_frequent',
                #     'fill_value': mode_value,
                #     'most_common_frequency': most_common_freq * 100,  # as percentage
                #     'missing_count': self.df[col].isnull().sum(),
                #     'missing_percentage': (self.df[col].isnull().sum() / len(self.df)) * 100,
                #     'unique_values': self.df[col].nunique()
                # }
        
        # # Add summary statistics
        # imputation_strategies['summary'] = {
        #     'total_missing': self.df.isnull().sum().sum(),
        #     'total_missing_percentage': (self.df.isnull().sum().sum() / (len(self.df) * len(self.df.columns))) * 100,
        #     'columns_with_missing': self.df.isnull().sum()[self.df.isnull().sum() > 0].index.tolist()
        # }
        
        self.df = df_imputed.copy()
    """
    def advanced_missing_data_analysis(self):
        Perform advanced missing data analysis and visualization
        # Missing data patterns
        fig = go.Figure()

        # Missing data heatmap
        missing_matrix = self.df.isnull().astype(int)
        fig.add_trace(
            go.Heatmap(
                z=missing_matrix.T,
                x=range(len(missing_matrix)),
                y=missing_matrix.columns,
                colorscale='RdYlBu',
                name='Missing Pattern'
            )
        )

        fig.update_layout(
            title='Missing Data Pattern Analysis',
            xaxis_title='Row Index',
            yaxis_title='Features',
            height=600
        )
        fig.show()

        # Missing data correlation
        missing_corr = missing_matrix.corr()
        fig = go.Figure(data=go.Heatmap(
            z=missing_corr,
            x=missing_corr.columns,
            y=missing_corr.columns,
            colorscale='RdBu'
        ))
        fig.update_layout(
            title='Missing Data Correlation Analysis',
            height=600
        )
        fig.show()

        return missing_corr
    """
    
    def advanced_categorical_encoding(self, ordinal_columns=None, save_path='encoders'):
        """
        Encode categorical variables using appropriate encoding methods:
        - OneHotEncoder for ordinal variables if specified, otherwise LabelEncoder
        - LabelEncoder for all other nominal variables
    
        Parameters:
        -----------
        ordinal_columns : list, optional
            List of column names that should be treated as ordinal variables and encoded with OneHotEncoder
        save_path : str, default='encoders'
            Directory path to save the encoder models
    
        Returns:
        --------
        DataFrame : Encoded dataframe
        dict : Dictionary containing encoding information and statistics
        """
        if ordinal_columns is None:
            ordinal_columns = []
    
        # Create directory for saving encoders if it doesn't exist
        os.makedirs(save_path, exist_ok=True)
    
        # Initialize containers
        encoding_info = {}
        df_encoded = self.df.copy()
    
        # Process categorical columns
        for col in self.categorical_cols:
            n_unique = self.df[col].nunique()
            value_counts = self.df[col].value_counts()
            missing_count = self.df[col].isnull().sum()
    
            # Store basic column statistics
            encoding_info[col] = {
                'unique_values': n_unique,
                'value_distribution': value_counts.to_dict(),
                'missing_values': missing_count,
                'missing_percentage': (missing_count / len(self.df)) * 100
            }
    
            # Handle missing values before encoding
            if missing_count > 0:
                df_encoded[col] = df_encoded[col].fillna('Missing')
    
            # Determine encoding method based on ordinal_columns list
            if col in ordinal_columns:
                # Use OneHotEncoder for specified ordinal variables
                encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
                encoded_data = encoder.fit_transform(df_encoded[[col]])
    
                # Get feature names after encoding
                feature_names = encoder.get_feature_names_out([col])
    
                # Replace original column with encoded columns
                df_encoded = df_encoded.drop(col, axis=1)
                encoded_df = pd.DataFrame(
                    encoded_data,
                    columns=feature_names,
                    index=df_encoded.index
                )
                df_encoded = pd.concat([df_encoded, encoded_df], axis=1)
    
                # Store encoding information
                encoding_info[col].update({
                    'encoding_type': 'onehot',
                    'encoded_features': feature_names.tolist(),
                    'categories': encoder.categories_[0].tolist()
                })
    
            else:
                # Use LabelEncoder for all other categorical columns
                encoder = LabelEncoder()
                df_encoded[col] = encoder.fit_transform(df_encoded[col])
    
                # Store mapping information
                encoding_info[col].update({
                    'encoding_type': 'label',
                    'mapping': dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
                })
    
            # Save each encoder individually
            encoder_filename = os.path.join(save_path, f'{col}_encoder.pkl')
            with open(encoder_filename, 'wb') as f:
                pickle.dump(encoder, f)
    
            # Add encoder file path to encoding info
            encoding_info[col]['encoder_path'] = encoder_filename
    
        # Add metadata information
        encoding_info['metadata'] = {
            'total_features_after_encoding': len(df_encoded.columns),
            'ordinal_columns': ordinal_columns,
            'nominal_columns': [col for col in self.categorical_cols if col not in ordinal_columns]
        }
    
        return df_encoded, encoding_info
    def advanced_missing_data_analysis(self):
        # Create a heatmap of missing data
        missing_matrix = self.df.isnull().astype(int)

        # Create a heatmap
        fig = go.Figure()
        fig.add_trace(
            go.Heatmap(
                z=missing_matrix.T,  # Use the transposed matrix for the heatmap
                x=list(missing_matrix.columns),  # Convert to list for x
                y=list(missing_matrix.index),     # Convert to list for y
                colorscale='Viridis',
                colorbar=dict(title='Missing Value Indicator')
            )
        )
        fig.update_layout(title='Missing Data Heatmap')
        fig.show()
        
    def advanced_imputation(self):
        """Perform advanced imputation using multiple methods for both numerical and categorical data"""
        df_imputed = {}
        
        # Separate numerical and categorical columns
        numerical_cols = self.df.select_dtypes(include=['number']).columns
        categorical_cols = self.df.select_dtypes(exclude=['number']).columns
        print(numerical_cols)
        print(categorical_cols)
        # 1. Iterative Imputer (MICE) for numerical data
        
        # imp_iter = IterativeImputer(
        #     estimator=RandomForestRegressor(n_estimators=100),
        #     max_iter=10,
        #     random_state=42
        # )
        # df_num_imputed = pd.DataFrame(
        #     imp_iter.fit_transform(self.df[numerical_cols]),
        #     columns=numerical_cols
        # )
        
        # Mode Imputer for categorical data
        if (len(categorical_cols) >0):
            imp_mode = SimpleImputer(strategy='most_frequent')
            df_cat_imputed = pd.DataFrame(
                imp_mode.fit_transform(self.df[categorical_cols]),
                columns=categorical_cols
            )
        
        # Combine numerical and categorical imputations
        
        # df_imputed['mice'] = pd.concat([df_num_imputed, df_cat_imputed], axis=1)
    
        # 2. KNN Imputer with optimal k for numerical data
        #k_values = range(1, min(20, int(len(self.df)*0.001)))
        k_values = range(1, 20)
        scores = []
    
        for k in k_values:
            imp_knn = KNNImputer(n_neighbors=k)
            imputed = imp_knn.fit_transform(self.df[numerical_cols])
            scores.append(np.mean(np.abs(imputed - self.df[numerical_cols].fillna(self.df[numerical_cols].mean()))))
    
        optimal_k = k_values[np.argmin(scores)]
        imp_knn = KNNImputer(n_neighbors=optimal_k)
        df_num_imputed_knn = pd.DataFrame(
            imp_knn.fit_transform(self.df[numerical_cols]),
            columns=numerical_cols
        )
        
        # Reuse mode imputer results for categorical data in KNN version
        if (len(categorical_cols) >0):
            df_imputed['knn'] = pd.concat([df_num_imputed_knn, df_cat_imputed], axis=1)
        df_imputed['knn'] = pd.concat([df_num_imputed_knn], axis=1)
        # 3. Matrix Factorization for numerical data only (SVD)
        
        # U, s, Vh = np.linalg.svd(self.df[numerical_cols].fillna(0), full_matrices=False)
        # rank = 3  # Can be adjusted based on explained variance
        # df_num_imputed_svd = pd.DataFrame(
        #     np.dot(U[:, :rank] * s[:rank], Vh[:rank, :]),
        #     columns=numerical_cols
        # )
        
        # # Combine SVD-imputed numerical data with mode-imputed categorical data
        # df_imputed['svd'] = pd.concat([df_num_imputed_svd, df_cat_imputed], axis=1)
    
        self.df = df_imputed['knn'].copy()
    def advanced_outlier_detection(self):
        """Perform advanced outlier detection"""
        outliers = {}

        # 1. Isolation Forest
        iso_forest = IsolationForest(contamination=0.1, random_state=42)
        outliers['isolation_forest'] = iso_forest.fit_predict(self.df[self.numeric_cols])

        # 2. Modified Z-score
        def modified_zscore(x):
            median = np.median(x)
            mad = np.median(np.abs(x - median))
            modified_zscores = 0.6745 * (x - median) / mad
            return np.abs(modified_zscores) > 3.5

        outliers['modified_zscore'] = self.df[self.numeric_cols].apply(modified_zscore)

        # 3. Cook's Distance for regression
        if self.target_col:


            X = self.df[self.numeric_cols].drop(columns=[self.target_col])
            y = self.df[self.target_col]
            model = sm.OLS(y, sm.add_constant(X)).fit()
            influence = OLSInfluence(model)
            outliers['cooks_distance'] = influence.cooks_distance[0] > 4/len(self.df)

        return outliers

    def relationship_analysis(self):
        return AdvancedRelationshipAnalysis.analyze_relationships(self.df)
        
    def dimensionality_reduction_analysis(self):
        """Perform advanced dimensionality reduction analysis"""
        # Prepare data
        scaler = RobustScaler()
        scaled_data = scaler.fit_transform(self.df[self.numeric_cols])

        # 1. PCA with explained variance analysis
        pca = PCA()
        pca_result = pca.fit_transform(scaled_data)

        # Explained variance plot
        # Explained variance plot
        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=list(range(1, len(pca.explained_variance_ratio_) + 1)),
                y=np.cumsum(pca.explained_variance_ratio_),
                mode='lines+markers',
                name='Cumulative Explained Variance'
            )
        )
        fig.update_layout(
            title='PCA Explained Variance Analysis',
            xaxis_title='Number of Components',
            yaxis_title='Cumulative Explained Variance Ratio'
        )
        fig.show()

        # 2. t-SNE visualization
        tsne = TSNE(n_components=2, random_state=42)
        tsne_result = tsne.fit_transform(scaled_data)

        fig = go.Figure(data=go.Scatter(
            x=tsne_result[:, 0],
            y=tsne_result[:, 1],
            mode='markers',
            marker=dict(
                color=self.df[self.target_col] if self.target_col else None,
                colorscale='Viridis'
            )
        ))
        fig.update_layout(title='t-SNE Visualization')
        fig.show()

        # 3. UMAP visualization
        umap_model = umap.UMAP(random_state=42)
        umap_result = umap_model.fit_transform(scaled_data)

        fig = go.Figure(data=go.Scatter(
            x=umap_result[:, 0],
            y=umap_result[:, 1],
            mode='markers',
            marker=dict(
                color=self.df[self.target_col] if self.target_col else None,
                colorscale='Viridis'
            )
        ))
        fig.update_layout(title='UMAP Visualization')
        fig.show()

    def advanced_categorical_analysis(self):
        """Perform advanced categorical data analysis"""
        if len(self.categorical_cols) > 0:
            # 1. Multiple Correspondence Analysis
            mca = prince.MCA(n_components=2)
            mca_coords = mca.fit_transform(self.df[self.categorical_cols])

            fig = go.Figure(data=go.Scatter(
                x=mca_coords[0],
                y=mca_coords[1],
                mode='markers',
                text=self.df.index,
                hoverinfo='text'
            ))
            fig.update_layout(title='Multiple Correspondence Analysis')
            fig.show()

            # 2. Advanced categorical encoding comparison
            encoders = {
                'target': TargetEncoder(),
                'woe': WOEEncoder(),
                'catboost': CatBoostEncoder()
            }

            encoded_dfs = {}
            if self.target_col:
                for name, encoder in encoders.items():
                    encoded_dfs[name] = encoder.fit_transform(
                        self.df[self.categorical_cols],
                        self.df[self.target_col]
                    )

            # Compare encodings
            if encoded_dfs:
                for col in self.categorical_cols:
                    fig = go.Figure()
                    for name, encoded_df in encoded_dfs.items():
                        fig.add_trace(
                            go.Box(
                                y=encoded_df[col],
                                name=name,
                                boxpoints='all'
                            )
                        )
                    fig.update_layout(
                        title=f'Encoding Comparison for {col}',
                        yaxis_title='Encoded Values'
                    )
                    fig.show()

    def advanced_distribution_analysis(self):
        """Perform advanced distribution analysis with interactive plots"""
        distribution_plots = {}
        for col in self.numeric_cols:
            # Create distribution plot with multiple components
            fig = make_subplots(rows=2, cols=2,
                              subplot_titles=['Distribution', 'Q-Q Plot',
                                            'Box Plot', 'Violin Plot'])
            
            data = self.df[col].dropna()
            
            # Histogram with optional KDE
            try:
                # Only attempt KDE if we have enough unique values
                if len(data.unique()) > 1:
                    # Add some small random noise if all values are integers
                    if data.dtype.kind in 'iu' or (data % 1 == 0).all():
                        data_for_kde = data + np.random.normal(0, 0.01, len(data))
                    else:
                        data_for_kde = data
                        
                    kde = stats.gaussian_kde(data_for_kde)
                    x_range = np.linspace(min(data), max(data), 100)
                    
                    # Add histogram
                    fig.add_trace(
                        go.Histogram(x=data, name='Histogram',
                                   nbinsx=50, histnorm='probability density'),
                        row=1, col=1
                    )
                    
                    # Add KDE curve
                    fig.add_trace(
                        go.Scatter(x=x_range, y=kde(x_range), name='KDE'),
                        row=1, col=1
                    )
                else:
                    # Fallback to simple histogram if KDE is not possible
                    fig.add_trace(
                        go.Histogram(x=data, name='Histogram',
                                   nbinsx=50),
                        row=1, col=1
                    )
            except (LinAlgError, ValueError) as e:
                # Fallback to simple histogram if KDE fails
                fig.add_trace(
                    go.Histogram(x=data, name='Histogram',
                               nbinsx=50),
                    row=1, col=1
                )
                
            # Q-Q Plot with error handling
            try:
                qq = stats.probplot(data)
                fig.add_trace(
                    go.Scatter(x=qq[0][0], y=qq[0][1],
                              mode='markers', name='Q-Q'),
                    row=1, col=2
                )
                # Add reference line
                z = np.array([min(qq[0][0]), max(qq[0][0])])
                fig.add_trace(
                    go.Scatter(x=z, y=z, mode='lines',
                              name='Reference Line',
                              line=dict(dash='dash')),
                    row=1, col=2
                )
            except Exception as e:
                # Add empty plot with error message if Q-Q plot fails
                fig.add_annotation(text="Unable to create Q-Q plot",
                                 xref="x2", yref="y2",
                                 x=0.5, y=0.5, showarrow=False)
            
            # Box Plot
            fig.add_trace(
                go.Box(y=data, name='Box Plot',
                      boxpoints='outliers'),  # Show outliers
                row=2, col=1
            )
            
            # Violin Plot with error handling
            try:
                fig.add_trace(
                    go.Violin(y=data, name='Violin Plot',
                             box_visible=True, meanline_visible=True,
                             points='outliers'),  # Show outliers
                    row=2, col=2
                )
            except Exception as e:
                # Fallback to box plot if violin plot fails
                fig.add_trace(
                    go.Box(y=data, name='Box Plot',
                          boxpoints='outliers'),
                    row=2, col=2
                )
            
            # Update layout with additional statistical information
            stats_text = (
                f"Mean: {data.mean():.2f}<br>"
                f"Median: {data.median():.2f}<br>"
                f"Std Dev: {data.std():.2f}<br>"
                f"Skewness: {data.skew():.2f}<br>"
                f"Kurtosis: {data.kurtosis():.2f}"
            )
            
            fig.update_layout(
                height=800,
                title=f'Distribution Analysis: {col}',
                annotations=[
                    dict(
                        x=1.0,
                        y=1.0,
                        xref='paper',
                        yref='paper',
                        text=stats_text,
                        showarrow=False,
                        align='left'
                    )
                ]
            )
            
            distribution_plots[col] = fig
            
        return distribution_plots
    def advanced_feature_relationships(self):
        """Analyze feature relationships using advanced techniques"""
        feature_relationship_plots = {}
    
        # 1. Advanced Correlation Analysis
        if len(self.numeric_cols) > 1:
            # Spearman and Kendall correlations
            correlations = {
                'pearson': self.df[self.numeric_cols].corr(method='pearson'),
                'spearman': self.df[self.numeric_cols].corr(method='spearman'),
                'kendall': self.df[self.numeric_cols].corr(method='kendall')
            }
            # Plot correlation differences
            for method, corr_matrix in correlations.items():
                fig = go.Figure(data=go.Heatmap(
                    z=corr_matrix,
                    x=corr_matrix.columns,
                    y=corr_matrix.columns,
                    colorscale='RdBu'
                ))
                fig.update_layout(
                    title=f'{method.capitalize()} Correlation Matrix',
                    height=600
                )
                feature_relationship_plots[f'{method.capitalize()} Correlation Matrix'] = fig
    
        # 2. Non-linear Relationships
        if len(self.numeric_cols) > 1:
            for i, col1 in enumerate(self.numeric_cols[:-1]):
                for col2 in self.numeric_cols[i+1:]:
                    fig = go.Figure()
                    
                    # Remove rows with NaN or infinite values
                    df_clean = self.df[[col1, col2]].replace([np.inf, -np.inf], np.nan).dropna()
                    
                    # Scatter plot
                    fig.add_trace(
                        go.Scatter(
                            x=df_clean[col1],
                            y=df_clean[col2],
                            mode='markers',
                            name='Data Points'
                        )
                    )
                    
                    try:
                        # Add LOWESS smoothing
                        lowess = sm.nonparametric.lowess
                        z = lowess(df_clean[col2], df_clean[col1])
                        fig.add_trace(
                            go.Scatter(
                                x=z[:, 0],
                                y=z[:, 1],
                                name='LOWESS',
                                line=dict(color='red')
                            )
                        )
                    except FloatingPointError:
                        print(f"Floating point error in LOWESS for {col1} vs {col2}. Skipping LOWESS.")
    
                    fig.update_layout(
                        title=f'Non-linear Relationship: {col1} vs {col2}',
                        xaxis_title=col1,
                        yaxis_title=col2,
                        height=600
                    )
                    feature_relationship_plots[f'{col1} vs {col2}'] = fig
    
        return feature_relationship_plots

    def generate_report(self):
        """Generate comprehensive analysis report"""
        # Create Sweetviz report
        # report = sv.analyze(self.df)
        # report.show_html('advanced_eda_report.html')
    
        # # Create DataPrep report
        # dpeda_report = create_report(self.df)
        # dpeda_report.show_browser()
    
        # Generate advanced distribution and feature relationship plots
        distribution_plots = self.advanced_distribution_analysis()
        feature_relationship_plots = self.advanced_feature_relationships()
    
        # Save the distribution plots to a separate HTML page
        with open('distribution_plots.html', 'w') as f:
            f.write("""
                <html>
                    <head>
                        <title>Distribution Plots</title>
                    </head>
                    <body>
            """)
            for plot_title, plot in distribution_plots.items():
                f.write(f"<h2>{plot_title}</h2>")
                f.write(plot.to_html(include_plotlyjs='cdn', full_html=False))
            f.write("""
                    </body>
                </html>
            """)
    
        # Save the feature relationship plots to a separate HTML page
        with open('feature_relationship_plots.html', 'w') as f:
            f.write("""
                <html>
                    <head>
                        <title>Feature Relationship Plots</title>
                    </head>
                    <body>
            """)
            for plot_title, plot in feature_relationship_plots.items():
                f.write(f"<h2>{plot_title}</h2>")
                f.write(plot.to_html(include_plotlyjs='cdn', full_html=False))
            f.write("""
                    </body>
                </html>
            """)
    
        # Summary statistics and insights
        summary = {
            'dataset_shape': self.df.shape,
            'missing_percentages': (self.df.isnull().sum() / len(self.df) * 100).to_dict(),
            'numeric_features': len(self.numeric_cols),
            'categorical_features': len(self.categorical_cols),
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
    
        # Create the main report page
        with open('advanced_eda_report.html', 'a') as f:
            f.write("""
                <html>
                    <head>
                        <title>Advanced EDA Report</title>
                    </head>
                    <body>
                        <h1>Advanced EDA Report</h1>
                        <h2>Summary</h2>
                        <pre>{}</pre>
                        <h2>Distribution Plots</h2>
                        <iframe src="distribution_plots.html" width="100%" height="2000"></iframe>
                        <h2>Feature Relationship Plots</h2>
                        <iframe src="feature_relationship_plots.html" width="100%" height="2000"></iframe>
                    </body>
                </html>
            """.format(json.dumps(summary, indent=4)))
    
        return summary
    # def run_all_eda(self):
    #     self.advanced_missing_data_analysis()
    #     self.advanced_imputation()
    #     self.advanced_outlier_detection()
    #     self.advanced_distribution_analysis()
    #     self.advanced_feature_relationships()
    #     self.dimensionality_reduction_analysis()
    #     self.advanced_categorical_analysis()
    #     self.generate_report()

In [13]:
df = pd.read_csv("./data.csv")
eda = AdvancedEDA(df, target_col="HasDetections")

In [14]:
eda.simple_statistical_imputation()

Index(['MachineIdentifier', 'ProductName', 'EngineVersion', 'AppVersion',
       'AvSigVersion', 'IsBeta', 'RtpStateBitfield', 'IsSxsPassiveMode',
       'DefaultBrowsersIdentifier', 'AVProductStatesIdentifier',
       'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm',
       'CountryIdentifier', 'CityIdentifier', 'OrganizationIdentifier',
       'GeoNameIdentifier', 'LocaleEnglishNameIdentifier', 'Platform',
       'Processor', 'OsVer', 'OsBuild', 'OsSuite', 'OsPlatformSubRelease',
       'OsBuildLab', 'SkuEdition', 'IsProtected', 'AutoSampleOptIn', 'PuaMode',
       'SMode', 'IeVerIdentifier', 'SmartScreen', 'Firewall', 'UacLuaenable',
       'Census_MDC2FormFactor', 'Census_DeviceFamily',
       'Census_OEMNameIdentifier', 'Census_OEMModelIdentifier',
       'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier',
       'Census_ProcessorModelIdentifier', 'Census_ProcessorClass',
       'Census_PrimaryDiskTotalCapacity', 'Census_PrimaryDiskTypeName',
       'Census_

In [15]:
joblib.dump(eda.df.columns,"clean_df_cols.pkl")

['clean_df_cols.pkl']

In [ ]:
eda.df.to_csv("./data_clean_0.csv" , index=False)

In [19]:
df_encoded, _ = eda.advanced_categorical_encoding()

In [21]:
X = df_encoded.drop("HasDetections",axis = 1)
cols = X.columns
sc = StandardScaler()
X = pd.DataFrame(sc.fit_transform(X) , columns=cols)
joblib.dump(sc,"StandardScaler.pkl")
X_train , X_test , y_train , y_test = train_test_split(X ,df_encoded.HasDetections , test_size = 0.2 , stratify=df_encoded.HasDetections )

In [ ]:
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/yahiabouzaouach/mlops_project.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']= 'bilel9999'
os.environ["MLFLOW_TRACKING_PASSWORD"] = '4a3b567f61acebf8c9bcd0cd17a4ce8d0e0a7cca'

In [ ]:
mlflow.set_tracking_uri('https://dagshub.com/yahiabouzaouach/mlops_project.mlflow') #your mlfow tracking uri
mlflow.set_experiment("Malware_detection")

In [ ]:
mlflow.sklearn.autolog(disable=True)

In [ ]:
with mlflow.start_run(run_name='LogisticRegression'):
    # mlflow.log_param("data_url",data_url)
    mlflow.log_param("data_version",0)
    mlflow.log_param("input_rows",eda.df.shape[0])
    mlflow.log_param("input_cols",eda.df.shape[1])
    #model fitting and training
    lr=LogisticRegression()
    mlflow.set_tag(key= "model",value="LogisticRegression")
    params = lr.get_params()
    mlflow.log_params(params)
    lr.fit(X_train,y_train)
    train_features_name = f'{X_train=}'.split('=')[0]
    train_label_name = f'{y_train=}'.split('=')[0]
    mlflow.set_tag(key="train_features_name",value= train_features_name)
    mlflow.set_tag(key= "train_label_name",value=train_label_name)
    predicted=lr.predict(X_test)
    precision,recall,fscore,support=score(y_test,predicted,average='macro')
    mlflow.log_metric("Precision_test",precision)
    mlflow.log_metric("Recall_test",recall)
    mlflow.log_metric("F1_score_test",fscore)
    mlflow.sklearn.log_model(lr,artifact_path="ML_models")

In [ ]:
with mlflow.start_run(run_name='RandomForestClassifier'):
    # mlflow.log_param("data_url",data_url)
    mlflow.log_param("data_version",0)
    mlflow.log_param("input_rows",eda.df.shape[0])
    mlflow.log_param("input_cols",eda.df.shape[1])
    #model fitting and training
    rf= RandomForestClassifier()
    mlflow.set_tag(key= "model",value="RandomForestClassifier")
    params = rf.get_params()
    mlflow.log_params(params)
    rf.fit(X_train,y_train)
    train_features_name = f'{X_train=}'.split('=')[0]
    train_label_name = f'{y_train=}'.split('=')[0]
    mlflow.set_tag(key="train_features_name",value= train_features_name)
    mlflow.set_tag(key= "train_label_name",value=train_label_name)
    predicted=lr.predict(X_test)
    precision,recall,fscore,support=score(y_test,predicted,average='macro')
    mlflow.log_metric("Precision_test",precision)
    mlflow.log_metric("Recall_test",recall)
    mlflow.log_metric("F1_score_test",fscore)
    mlflow.sklearn.log_model(rf,artifact_path="ML_models")

In [ ]:
def objective(trial):
    """
    Optuna objective function for hyperparameter optimization
    """

    params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
    'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.3),
    'num_leaves': trial.suggest_int('num_leaves', 31, 200),
    'max_depth': trial.suggest_int('max_depth', 5, 30),
    'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
    'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
    'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
    'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
    'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
    'device': 'gpu',  # Use GPU
    'gpu_platform_id': 0,
    'gpu_device_id': 0,
    }

    # Create pipeline with model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', lgb.LGBMClassifier(**params, random_state=42))
    ])
    
    # Perform cross-validation
    try:
        cv_scores = cross_val_score(
            pipeline, 
            X_train, 
            y_train, 
            cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
            scoring='roc_auc'
        )
        return np.mean(cv_scores)
    except Exception as e:
        print(f"Error in trial: {e}")
        return np.nan

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)  # Reduced trials for quicker demonstration

# Print best trial information
print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('  Value: ', trial.value)
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

# Train final model with best hyperparameters
best_params = trial.params
best_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', lgb.LGBMClassifier(**best_params, random_state=42))
])

In [ ]:
with mlflow.start_run(run_name='LightGBM'):
    mlflow.log_param("data_version",0)
    mlflow.log_param("input_rows",eda.df.shape[0])
    mlflow.log_param("input_cols",eda.df.shape[1])
    #model fitting and training
    mlflow.set_tag(key= "model",value="LightGBM")
    params = best_pipeline.get_params()
    mlflow.log_params(params)
    best_pipeline.fit(X_train, y_train)
    train_features_name = f'{X_train=}'.split('=')[0]
    train_label_name = f'{y_train=}'.split('=')[0]
    mlflow.set_tag(key="train_features_name",value= train_features_name)
    mlflow.set_tag(key= "train_label_name",value=train_label_name)
    predicted=best_pipeline.predict_proba(X_test)[:, 1]
    binary_predictions = (predicted >= 0.5).astype(int)
    mlflow.log_metric("Precision_test",precision_score(y_test, binary_predictions))
    mlflow.log_metric("Recall_test",recall_score(y_test, binary_predictions))
    mlflow.log_metric("F1_score_test",f1_score(y_test, binary_predictions))
    mlflow.log_metric("roc_auc_score", roc_auc_score(y_test, binary_predictions))
    mlflow.sklearn.log_model(best_pipeline,artifact_path="ML_models")